## API Key Set up

In [21]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI API KEY IS NOT FOUND IN THE ENVIORMENT VARIABLES")

In [22]:
from agents import Agent, Runner
from pydantic import BaseModel
from agents import Agent


## A simple agent

In [ ]:
agent = Agent(
    name = "Basic Agent",
    instructions= "You are a helpful assistant. Respond on in all caps."
)

result = await Runner.run(agent, "Hello! How are you")

In [8]:
result

"""
RunResult(input='Hello! How are you', 
        new_items=[MessageOutputItem(agent=Agent(name='Basic Agent', 
                                                 instructions='You are a helpful assistant. Respond on in all caps.', 
                                                 prompt=None, 
                                                 handoff_description=None, 
                                                 handoffs=[], 
                                                 model=None, 
                                                 model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, 
                                                                              presence_penalty=None, tool_choice=None, parallel_tool_calls=None, 
                                                                              truncation=None, max_tokens=None, reasoning=None, metadata=None, 
                                                                              store=None, include_usage=None, response_include=None, 
                                                                              extra_query=None, extra_body=None, extra_headers=None, extra_args=None), 
                                                 tools=[], 
                                                 mcp_servers=[], 
                                                 mcp_config={}, 
                                                 input_guardrails=[], 
                                                 output_guardrails=[], 
                                                 output_type=None, 
                                                 hooks=None, 
                                                 tool_use_behavior='run_llm_again', 
                                                 reset_tool_choice=True), 
                                                 raw_item=ResponseOutputMessage(id='msg_686bc4b96804819aa9a00ff73018fdf708fab89c767a5d68', 
                                                                                content=[ResponseOutputText(annotations=[], 
                                                                                                            text="HELLO! I'M HERE AND READY TO ASSIST YOU. HOW CAN I HELP TODAY?", 
                                                                                                            ype='output_text', 
                                                                                                            logprobs=[])], 
                                                                                role='assistant', 
                                                                                status='completed', 
                                                                                type='message'), 
                                                                                type='message_output_item')], 
        raw_responses=[ModelResponse(output=[ResponseOutputMessage(id='msg_686bc4b96804819aa9a00ff73018fdf708fab89c767a5d68', 
                                                                   content=[ResponseOutputText(annotations=[], 
                                                                                               text="HELLO! I'M HERE AND READY TO ASSIST YOU. HOW CAN I HELP TODAY?", 
                                                                                               type='output_text', 
                                                                                               logprobs=[])], 
                                                                    role='assistant', 
                                                                    status='completed', 
                                                                    type='message')], 
                                    usage=Usage(requests=1, input_tokens=28, 
                                                input_tokens_details=InputTokensDetails(cached_tokens=0), 
                                                output_tokens=20, 
                                                output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=48), 
                                                response_id='resp_686bc4b878f8819a9c18545de6caf40108fab89c767a5d68')], 
        final_output="HELLO! I'M HERE AND READY TO ASSIST YOU. HOW CAN I HELP TODAY?", input_guardrail_results=[], output_guardrail_results=[], context_wrapper=RunContextWrapper(context=None, usage=Usage(requests=1, input_tokens=28, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=20, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=48)), _last_agent=Agent(name='Basic Agent', instructions='You are a helpful assistant. Respond on in all caps.', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True))

"""

RunResult(input='Hello! How are you', new_items=[MessageOutputItem(agent=Agent(name='Basic Agent', instructions='You are a helpful assistant. Respond on in all caps.', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_686bc4b96804819aa9a00ff73018fdf708fab89c767a5d68', content=[ResponseOutputText(annotations=[], text="HELLO! I'M HERE AND READY TO ASSIST YOU. HOW CAN I HELP TODAY?", type='output_text', logprobs=[])], role='assis

In [9]:
result.final_output

"HELLO! I'M HERE AND READY TO ASSIST YOU. HOW CAN I HELP TODAY?"

In [10]:
joke_agent = Agent(
    name = "Joke Agent",
    instructions= "You are a joke teller. You are given a topic and you need to tell a joke about it."
)

topic = "Trump"

result = await Runner.run(joke_agent, "Trump")
result.final_output

'Why did Donald Trump sit on a lightbulb?\n\nHe wanted to keep his promises bright!'

## Multi-Agent Communication

In [15]:
language_agent = Agent(
    name = "Language Agent",
    instructions= "You are a language expert. You are given a joke and you need to rewrite it in a different language."
)

topic = "Trump"


joke_result = await Runner.run(joke_agent, "Trump")
translated_result = await Runner.run(language_agent, f"Translate this joke to Chinese: {joke_result.final_output}")
print(joke_result.final_output)
print(translated_result.final_output)

Why does Donald Trump always carry a golf pencil?  

In case he has to draw a red line!
为什么唐纳德·特朗普总是带着高尔夫铅笔？

以防他需要画红线！


## Structual Output

In [18]:
class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int
    servings: int


recipe_agent = Agent(
    name = "recipe_agent",
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job"
                  "is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "Stake")

title='Garlic Butter Herb Steak' ingredients=['2 ribeye steaks', '2 tablespoons olive oil', '4 cloves garlic, minced', '3 tablespoons unsalted butter', '1 tablespoon fresh rosemary, chopped', '1 tablespoon fresh thyme, chopped', 'Salt and pepper to taste'] cooking_time=25 servings=2


In [19]:
print(response.final_output)

title='Garlic Butter Herb Steak' ingredients=['2 ribeye steaks', '2 tablespoons olive oil', '4 cloves garlic, minced', '3 tablespoons unsalted butter', '1 tablespoon fresh rosemary, chopped', '1 tablespoon fresh thyme, chopped', 'Salt and pepper to taste'] cooking_time=25 servings=2


In [20]:
print(response.final_output.title)
print(response.final_output.ingredients)
print(response.final_output.cooking_time)
print(response.final_output.servings)

Garlic Butter Herb Steak
['2 ribeye steaks', '2 tablespoons olive oil', '4 cloves garlic, minced', '3 tablespoons unsalted butter', '1 tablespoon fresh rosemary, chopped', '1 tablespoon fresh thyme, chopped', 'Salt and pepper to taste']
25
2


## Tool Calling


[TODO]: 怎么人为干预agent选择 tool 的流程

[TODO]: Other basic Tools

In [29]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    print("get_weather is triggered")
    return f"The weather in {city} is sunny."

@function_tool
def get_temp(city: str) -> str:
    print("get_temp is triggered")
    return f"The temp in {city} is 35."

@function_tool
def add(x: int, y: int) -> int:
    print("add is triggered")
    return x + y

agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather, get_temp, add],
)

result = await Runner.run(agent, "What is the weather for Dallas?")

get_weather is triggered


In [28]:
print(result.final_output)

The weather in Dallas is sunny.


In [30]:
result = await Runner.run(agent, "What is the weather for Beijing and Xi'an? What is the sum of their temp?")

get_weather is triggered
get_weather is triggered
get_temp is triggered
get_temp is triggered
add is triggered


In [31]:
result.final_output

"The weather in both Beijing and Xi'an is sunny. The temperature in Beijing is 35°C, and in Xi'an, it's also 35°C. The sum of their temperatures is 70°C."

In [33]:
from agents import WebSearchTool

news_agent = Agent(
    name = "News Resporter",
    instructions= "You are a new Reporter. Your job is to find recent news articales on the internet about China politics",
    tools = [WebSearchTool()]
)

In [34]:
result = await Runner.run(news_agent, "find news")
print(result.final_output)

Here are some recent developments in Chinese politics:

**Leadership and Governance**

- **Expulsion of Former Defense Ministers**: In June 2024, the Chinese Communist Party expelled former Defense Minister Li Shangfu and his predecessor, Wei Fenghe, on corruption charges. Li was dismissed in October 2023 after a two-month public absence. His case has been referred to military prosecutors, potentially leading to a life sentence. ([aljazeera.com](https://www.aljazeera.com/news/2024/6/27/chinas-communist-party-expels-ex-defence-ministers-over-corruption-charges?utm_source=openai))

- **National People's Congress (NPC) 2024**: In March 2024, the NPC set a 5% economic growth target and announced a 7.2% increase in defense spending. The government emphasized stability amid economic challenges, including a real estate crisis and slowing growth. The report also adopted a firmer stance on Taiwan, omitting the term "peaceful" in reference to reunification efforts. ([apnews.com](https://apnews.c

## Handoffs

In [42]:
from agents import Agent, Runner, RunContextWrapper, handoff


def on_history_handoff(ctx: RunContextWrapper[None]):
    print("hannding off to history tutor [1]")

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.")

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("hannding off to math tutor [1]")

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples")

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
                "If neither agent is relevant, provide a general response.", 
    handoffs=[history_tutor_agent,
              handoff(math_tutor_agent, on_handoff=on_math_handoff),]
)

In [43]:
result = await Runner.run(triage_agent, "How do I calulate 2^3^3")
print(result.final_output)

hannding off to math tutor [1]
To calculate \(2^{3^3}\), you need to be mindful of the order of operations with exponents. Here's how to proceed:

1. **Evaluate the exponent from the top down.** First, solve the exponentiation in the upper part: \(3^3\).

   \[
   3^3 = 3 \times 3 \times 3 = 27
   \]

2. **Use the result in the next calculation.** Now substitute this result back into the expression for the larger calculation:

   \[
   2^{3^3} = 2^{27}
   \]

3. **Calculate \(2^{27}\).** This involves multiplying 2 by itself 27 times. Generally, a calculator is useful here since the number gets very large.

   \[
   2^{27} = 134,217,728
   \]

Therefore, \(2^{3^3}\) equals 134,217,728.
